In [2]:
import time
import numpy as np
import pandas as pd

import requests

In [9]:
# df = pd.read_excel(r'..\datasets\250716.xlsx')
# print(df.head())

df = pd.read_csv(r'..\datasets\df_na2.csv', encoding = 'cp949')
print(df.head())

    순번                 출발지 주소                  중간집결지 주소             최종 도착지 주소  \
0   73  서울특별시 마포구 대흥로24라길 6-8  서울특별시 동대문구 답십리로11길 30-22  강원특별자치도 원주시 북원로 3223   
1   82     경기도 안성시 대덕면 내리5길 1      경기도 수원시 영통구 월드컵로 206  경기도 수원시 영통구 월드컵로 206   
2  128  충청남도 천안시 동남구 수도산공원길 1  충청남도 천안시 동남구 천안대로 429-13      세종특별자치시  당산로 333   
3  278    충청남도 천안시 동남구 일봉로 17  충청남도 천안시 동남구 천안대로 429-13      세종특별자치시  당산로 333   
4  283  충청남도 천안시 서북구 부성18로 29  충청남도 천안시 동남구 천안대로 429-13      세종특별자치시  당산로 333   

     중간집결 기한    최종도착 기한 이동방법  start_lat   start_lng  mid_lat  mid_lng  \
0   0+1일 11시   0+1일 14시   집단  37.554816  126.948123      NaN      NaN   
1  0+14일 14시  0+14일 14시   개별  37.000611  127.223656      NaN      NaN   
2   0+6일 12시   0+6일 14시   집단  36.793048  127.151206      NaN      NaN   
3   0+6일 12시   0+6일 14시   집단  36.793980  127.129853      NaN      NaN   
4   0+6일 12시   0+6일 14시   집단  36.846130  127.150747      NaN      NaN   

     end_lat     end_lng                출발주소_수정             중간주소_수정  \
0  

In [24]:
num = np.array(df['순번'])

start = np.array(df['출발지 주소'])
mid = np.array(df['중간집결지 주소'])
end = np.array(df['최종 도착지 주소'])

# start = np.array(df['출발주소_수정'])
# mid = np.array(df['중간주소_수정'])
# end = np.array(df['최종주소_수정'])

In [ ]:
# def geocoding(address: list):
#     url = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"
    
#     headers = {
#         "x-ncp-apigw-api-key-id": "oa77i9oz1h",
#         "x-ncp-apigw-api-key": "xvC9h8wAZLXjsokASCSKjLfNJ5uR63sKBGz705KA",
#         "Accept": "application/json"
#     }

#     params = {"query": address}
#     resp = requests.get(url, headers = headers, params = params)
#     resp.raise_for_status()
#     data = resp.json()

#     if data.get('addresses'):
#         lat = float(data['addresses'][0]['y']) # 위도
#         lng = float(data['addresses'][0]['x']) # 경도  
#         return lat, lng
    
#     else:
#         lat = None
#         lng = None
#         return lat, lng

In [ ]:
def geocoding(address: list):
    url = "https://maps.apigw.ntruss.com/map-geocode/v2/geocode"
    
    headers = {
        "x-ncp-apigw-api-key-id": "oa77i9oz1h",
        "x-ncp-apigw-api-key": "xvC9h8wAZLXjsokASCSKjLfNJ5uR63sKBGz705KA",
        "Accept": "application/json"
    }

    params = {"query": address}
    resp = requests.get(url, headers = headers, params = params)
    resp.raise_for_status()
    data = resp.json()

    try:
        # if data.get('addresses'):
        lat = float(data['addresses'][0]['y']) # 위도
        lng = float(data['addresses'][0]['x']) # 경도  
        return lat, lng
    
    except Exception as e:
        print(f'{address} 변환 실패: {e}')

        lat = None
        lng = None
        return lat, lng


In [30]:
# geocoding
if __name__ == "__main__":
    chunk = 1_000
    start_index = 0
    last_index = 16_300
    
    for i in range(start_index, last_index, chunk):
        slc = slice(i, min(i + chunk, last_index))
        print(f'Processing records {slc.start} to {slc.stop}...')

        t0 = time.time()
        start_lat, start_lng = zip(*(geocoding(addr) for addr in start[slc]))
        mid_lat, mid_lng = zip(*(geocoding(addr) for addr in mid[slc]))
        end_lat, end_lng = zip(*(geocoding(addr) for addr in end[slc]))
        t1 = time.time()

        print(f'걸린시간(분): {(t1-t0)/60:.2f}')

        df_chunk = pd.DataFrame({'num': num[slc],
                       'start_lat': start_lat,
                       'start_lng': start_lng,
                       'mid_lat': mid_lat,
                       'mid_lng': mid_lng,
                       'end_lat': end_lat,
                       'end_lng': end_lng})
        
        mode = 'w' if i == start_index else 'a'
        header = True if i == start_index else False

        df_chunk.to_csv(r'..\results\transformed.csv', mode = mode, header = header, index = False)
        print(f'{slc.start} to {slc.stop} 저장 완료')

Processing records 0 to 1000...
충청남도 천안시 서북구 입장면 위례성로 1909-3 변환 실패: 'addresses'
대전광역시 대덕구 계족로690번길 135 변환 실패: list index out of range
서울특별시 동작구 등용로12길 25 변환 실패: list index out of range
대전광역시 유성구 둔곡길 58 변환 실패: list index out of range
경기도 남양주시 훈민로 300 변환 실패: list index out of range
광주광역시 광산구 신가번영로47번길 8-20 변환 실패: list index out of range
경기도 시흥시 마유로423번길 1042 변환 실패: list index out of range
충청남도 당진시 석문면 죽림로 45-25 변환 실패: list index out of range
경기도 고양시 덕양구 호국로1595번길 7676-6 변환 실패: list index out of range
경기도 의정부시 송양로 373-20 변환 실패: list index out of range
충청남도 서산시 대산읍 정자동2로 393-3 변환 실패: list index out of range
경상남도 창원시 성산구 대원로93번길 24 변환 실패: list index out of range
부산광역시 사상구 신천대로 34 변환 실패: list index out of range
경상북도 봉화군 상운면 하눌로 45 변환 실패: list index out of range
경기도 연천군 왕징면 왕산로 678-345 변환 실패: list index out of range
전라남도 영암군 시종면 고분로 287-60 변환 실패: list index out of range
서울특별시 중랑구 신내역로1길 20 변환 실패: list index out of range
경기도 양평군 용문면 용문로 25 변환 실패: list index out of range
서울특별시 동대문구 답십리로11길 30-2